### Joint Blind-Denoising and Cosmological Parameter Inference 

We create a alternating Gibbs Sampling procedure where we infer CMB signal (gaussian-like) parameters akin to noise in natural image denoising + posterior sampler is intersteller dust simulations

**CMB**  
Model temperature fluctuations as  
$$
\mu = 0,\quad \mathrm{Cov}(y) = \Sigma_\phi,
$$  

where  
$$
\Sigma_\phi = \Sigma(H_0,\;\Omega_b,\;\sigma).
$$  
Hubble Constant ~ Uniform-Sampling, Baryon Density ~ Uniform-Sampling, Noise Amplitude


##### Data-Loading + Inspection

In [ ]:
!pip install astropy camb healpy pixell

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import camb
from camb import model, initialpower
import pixell.enmap as enmap # For flat-sky map operations
from pixell import curvedsky # For generating GRFs from Cls
import healpy as hp # Often used by CAMB for Cls

ModuleNotFoundError: No module named 'camb'

In [ ]:
# --- Configuration ---
# Map properties
NPIX_SIDE = 256  # Number of pixels on a side
PIX_SIZE_ARCMIN = 8.0  # Pixel size in arcminutes
SHAPE, WCS = enmap.geometry(pos=(0,0), shape=(NPIX_SIDE, NPIX_SIDE), res=np.deg2rad(PIX_SIZE_ARCMIN/60.), proj="car")
# SHAPE will be (NPIX_SIDE, NPIX_SIDE), WCS is the world coordinate system object

# Cosmological Parameters (Fiducial values from paper's footnote 8)
H0_FID = 67.5 # Example, adjust if paper specifies exact CAMB H0
OMBH2_FID = 0.022 # Baryon density omega_b * h^2
OMCH2_FID = 0.122 # Cold dark matter density omega_c * h^2
OMK_FID = 0.0    # Omega_k
TAU_FID = 0.0544 # Optical depth
NS_FID = 0.9649  # Scalar spectral index
AS_FID = 2.1e-9  # Scalar amplitude (ln(10^10 As) = 3.044 => As ~ 2.1e-9)

# Noise parameter Phi = (sigma_cmb, H0_cosmo, ombh2_cosmo)
# Priors (from paper Section 3.2)
H0_PRIOR_MIN, H0_PRIOR_MAX = 50.0, 90.0
OMBH2_PRIOR_MIN, OMBH2_PRIOR_MAX = 0.0075, 0.0567 # Note: paper uses omega_b, CAMB uses ombh2
# To convert: omega_b = ombh2 / (H0/100)^2. For priors, it's easier to sample H0 and ombh2 directly.
SIGMA_CMB_PRIOR_MIN, SIGMA_CMB_PRIOR_MAX = 0.1, 1.2 # sigma_min should be >0. Let's use 0.1 for now.

# Data paths (MODIFY THESE)
DUST_DATA_DIR = "path/to/your/cats_dust_maps_256x256/" # Placeholder
OUTPUT_DIR = "generated_cosmo_data/"
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, "dust_maps"), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, "cmb_maps"), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, "mixed_maps"), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, "params"), exist_ok=True)

NUM_SAMPLES_TO_GENERATE = 10 # Small number for testing

Intersteller Dust Simulations (NEED: CATS data (real dust))

In [ ]:
def load_or_generate_dust_map(index, output_dir=os.path.join(OUTPUT_DIR, "dust_maps")):
    """
    Placeholder for loading a dust map.
    Replace this with actual loading from CATS database.
    For now, generates a synthetic "dust-like" map.
    """
    
    # --- BEGIN PLACHOLDER ---
    # Try to load if it exists (e.g., if you pre-process CATS)
    # filepath = os.path.join(DUST_DATA_DIR, f"dust_map_{index:04d}.npy")
    # if os.path.exists(filepath):
    #     return enmap.read_map(filepath) # Assuming maps are saved as .npy or FITS

    # Generate synthetic dust: power-law power spectrum
    ells = np.fft.fftfreq(NPIX_SIDE, d=np.deg2rad(PIX_SIZE_ARCMIN / 60.0)) * 2 * np.pi
    ell_grid_x, ell_grid_y = np.meshgrid(ells, ells)
    ell_mod = np.sqrt(ell_grid_x**2 + ell_grid_y**2)
    
    # Power law: P(k) ~ k^-alpha. Let's use alpha around 2.5-3 for dust-like structures
    alpha_dust = 2.7
    ps2d_dust = np.zeros_like(ell_mod)
    ps2d_dust[ell_mod > 0] = ell_mod[ell_mod > 0]**(-alpha_dust)
    ps2d_dust[0, 0] = 0 # No DC component in fluctuations

    # Generate random phases
    random_phases = np.exp(1j * 2 * np.pi * np.random.rand(NPIX_SIDE, NPIX_SIDE))
    
    # Create map in Fourier space
    map_fourier = np.sqrt(ps2d_dust) * random_phases
    
    # Inverse FFT to get real-space map
    dust_map_data = np.fft.ifft2(map_fourier).real
    
    # Normalize to have some reasonable range (e.g., mean 0, std 1, then scale)
    dust_map_data = (dust_map_data - np.mean(dust_map_data)) / np.std(dust_map_data)
    dust_map_data *= 50 # Arbitrary scaling for visual purposes
    
    dust_map = enmap.ndmap(dust_map_data, WCS)
    # --- END PLACEHOLDER ---

    # Save the generated/loaded map (optional, good for consistency)
    # enmap.write_map(os.path.join(output_dir, f"dust_{index:04d}.fits"), dust_map)
    
    return dust_map

In [ ]:
print("Testing dust map generation/loading...")
test_dust = load_or_generate_dust_map(0)
plt.figure(figsize=(5,5))
plt.imshow(test_dust, cmap='magma')
plt.title("Sample Synthetic Dust Map")
plt.colorbar()
# plt.savefig(os.path.join(OUTPUT_DIR,"sample_synthetic_dust.png"))
plt.show()
# print("Synthetic dust map generated and saved.")

In [ ]:
import numpy as np
from astropy.io import fits # Example for FITS files
import matplotlib.pyplot as plt
import os

https://www.mhdturbulence.com/

Cho-ENO - Mach ~ 7

In [ ]:
# --- Configuration ---
# This will depend on the exact data from CATS
# For example, if you download a snapshot:
SIMULATION_SNAPSHOT_PATH = "path/to/your/cats_simulation_snapshot.fits" # REPLACE
OUTPUT_DUST_MAP_DIR = "generated_dust_maps/"
os.makedirs(OUTPUT_DUST_MAP_DIR, exist_ok=True)

TARGET_MAP_SIZE = (256, 256) # As per papers

def create_column_density_map(density_cube_3d, axis=0):
    """
    Creates a 2D column density map by integrating along a specified axis.
    Assumes density_cube_3d is a 3D NumPy array.
    """
    if density_cube_3d.ndim != 3:
        raise ValueError("Input must be a 3D density cube.")
    
    # Sum along the specified axis. Multiplication by cell_depth can be done
    # if absolute physical units are needed, but for training a diffusion model
    # on these structures, relative variations are often most important.
    # The paper "Removing Dust..." Fig 1 caption says "global unit is arbitrary."
    column_density_map = np.sum(density_cube_3d, axis=axis)
    
    # Ensure output map is 256x256 (it should be if input cube is 256^3 and axis is correct)
    if column_density_map.shape != TARGET_MAP_SIZE:
        # This might happen if the downloaded cube isn't exactly 256^3 or if you need to select a slice
        print(f"Warning: Column density map shape is {column_density_map.shape}, expected {TARGET_MAP_SIZE}.")
        # Add cropping or resizing if necessary, though ideally, you select 256^3 cubes.
        # Example: simple center crop if larger
        # if column_density_map.shape[0] > TARGET_MAP_SIZE[0]:
        #     c_start0 = (column_density_map.shape[0] - TARGET_MAP_SIZE[0]) // 2
        #     column_density_map = column_density_map[c_start0:c_start0+TARGET_MAP_SIZE[0], :]
        # if column_density_map.shape[1] > TARGET_MAP_SIZE[1]:
        #     c_start1 = (column_density_map.shape[1] - TARGET_MAP_SIZE[1]) // 2
        #     column_density_map = column_density_map[:, c_start1:c_start1+TARGET_MAP_SIZE[1]]
        pass # For now, assume it's the correct size or handle later

    return column_density_map

# --- Example Usage (Conceptual - depends on your actual data file) ---
if os.path.exists(SIMULATION_SNAPSHOT_PATH):
    try:
        # Example: Assuming FITS file where density is in the primary HDU
        with fits.open(SIMULATION_SNAPSHOT_PATH) as hdul:
            density_data_3d = hdul[0].data # Adjust HDU index if needed
        
        print(f"Loaded 3D density cube with shape: {density_data_3d.shape}")

        if density_data_3d.shape == (TARGET_MAP_SIZE[0], TARGET_MAP_SIZE[1], TARGET_MAP_SIZE[2] if len(TARGET_MAP_SIZE)>2 else TARGET_MAP_SIZE[0]): # Assuming (256,256,256)
            # Create column density maps along different axes
            cd_map_z = create_column_density_map(density_data_3d, axis=0) # Integrate along 0th axis (e.g., z)
            cd_map_y = create_column_density_map(density_data_3d, axis=1) # Integrate along 1st axis (e.g., y)
            cd_map_x = create_column_density_map(density_data_3d, axis=2) # Integrate along 2nd axis (e.g., x)

            # Save and visualize one
            output_path = os.path.join(OUTPUT_DUST_MAP_DIR, "dust_map_z_axis_000.fits")
            fits.writeto(output_path, cd_map_z, overwrite=True)
            print(f"Saved column density map (z-axis) to {output_path}")

            plt.figure(figsize=(7,7))
            plt.imshow(cd_map_z, origin='lower', cmap='magma', vmax=np.percentile(cd_map_z, 99.5)) # vmax for better contrast
            plt.title("Generated Dust Column Density Map (z-axis integration)")
            plt.xlabel("Pixel X")
            plt.ylabel("Pixel Y")
            plt.colorbar(label="Column Density (Arbitrary Units)")
            plt.savefig(os.path.join(OUTPUT_DUST_MAP_DIR, "dust_map_z_axis_000.png"))
            plt.show()
            plt.close()

        else:
            print(f"3D data shape {density_data_3d.shape} not directly (256,256,256). Manual slicing/selection might be needed.")

    except Exception as e:
        print(f"Error processing simulation file: {e}")
        print("Please ensure SIMULATION_SNAPSHOT_PATH is correct and file is readable.")
        print("This script provides a template; actual data loading will depend on CATS file structure.")

else:
    print(f"Simulation snapshot not found at: {SIMULATION_SNAPSHOT_PATH}")
    print("Please download data from CATS (www.mhdturbulence.com) and update the path.")
    print("Generating a dummy 2D map for placeholder visualization...")
    
    # Placeholder dummy map if no real data
    dummy_dust_map = np.random.rand(TARGET_MAP_SIZE[0], TARGET_MAP_SIZE[1]) * 100
    plt.figure(figsize=(7,7))
    plt.imshow(dummy_dust_map, origin='lower', cmap='magma')
    plt.title("Dummy Dust Map (Placeholder)")
    plt.colorbar()
    plt.savefig(os.path.join(OUTPUT_DUST_MAP_DIR, "dummy_dust_map.png"))
    # plt.show() # Can be noisy
    plt.close()


# To generate 991 maps, you would loop through multiple snapshots
# and potentially different integration axes or apply rotations to the 3D cube
# before integration to get more variety if needed from a limited set of 3D snapshots.

CMB simulation

In [ ]:
def get_camb_cls(H0, ombh2, omch2=OMCH2_FID, omk=OMK_FID, tau=TAU_FID,
                 As=AS_FID, ns=NS_FID, lmax=3*NPIX_SIDE): # lmax somewhat larger than Nyquist
    """
    Uses CAMB to calculate TT power spectra.
    """
    pars = camb.CAMBparams()
    pars.set_cosmology(H0=H0, ombh2=ombh2, omch2=omch2, omk=omk, tau=tau)
    pars.InitPower.set_params(As=As, ns=ns, r=0) # r=0 for no tensors
    pars.set_for_lmax(lmax, lens_potential_accuracy=0) # lens_potential_accuracy=0 if not lensing

    results = camb.get_results(pars)
    powers = results.get_cmb_power_spectra(pars, CMB_unit='muK') # Get Cls in uK^2
    cl_tt = powers['total'][:, 0] # TT spectrum (0-indexed for l, so cl_tt[l] is C_l^TT)
    
    # CAMB returns Cls from l=0. We need l=0 to lmax.
    # Ensure cl_tt has length lmax+1. CAMB might return up to lmax_calc.
    if len(cl_tt) > lmax + 1:
        cl_tt = cl_tt[:lmax+1]
    elif len(cl_tt) < lmax + 1:
        # Pad with zeros if CAMB didn't compute up to lmax (shouldn't happen with set_for_lmax)
        cl_tt = np.pad(cl_tt, (0, lmax + 1 - len(cl_tt)), 'constant')

    # Remove monopole and dipole from Cls for map generation (often done)
    cl_tt[0] = 0 
    cl_tt[1] = 0
    return cl_tt # Units of uK^2

def generate_cmb_map(cl_tt, sigma_cmb_amp, seed=None):
    """
    Generates a flat-sky CMB map realization from Cls using pixell.
    cl_tt should be the power spectrum D_l = l(l+1)C_l/2pi or C_l.
    pixell.curvedsky.rand_map expects C_l.
    sigma_cmb_amp is the overall amplitude scaling factor mentioned in paper (Phi).
    """
    # The Cls from CAMB are C_l.
    # The sigma_cmb_amp from the paper seems to be a direct multiplier on the *covariance*,
    # so it's a multiplier on Cls (power), or on the map std. dev. if it's sqrt(power).
    # Let's assume sigma_cmb_amp scales the *standard deviation* of the CMB map.
    # So, C_l_scaled = (sigma_cmb_amp^2) * C_l_fiducial
    # However, the paper describes sigma as part of Phi, which parametrizes Sigma_Phi.
    # If Sigma_Phi = sigma^2 * Sigma_phi_base, then C_l_effective = sigma^2 * C_l_base
    
    scaled_cl_tt = cl_tt * (sigma_cmb_amp**2) # Scale power spectrum

    # pixell.curvedsky.rand_map needs an array of Cls [TT, EE, BB, TE, ...]
    # For TT only:
    cls_for_randmap = np.zeros((1, len(scaled_cl_tt))) # Shape (1, nl) for just T
    cls_for_randmap[0, :] = scaled_cl_tt
    
    cmb_map_data = curvedsky.rand_map(SHAPE, WCS, cls_for_randmap, seed=seed, pol=False) # pol=False for T only
    return enmap.ndmap(cmb_map_data, WCS) # Returns a single map (T)

In [ ]:
# Test CMB generation
print("\nTesting CMB map generation...")
fiducial_H0 = 67.5
fiducial_ombh2 = 0.02237 # Example value
fiducial_sigma_cmb = 1.0 

test_cls_tt = get_camb_cls(H0=fiducial_H0, ombh2=fiducial_ombh2)
test_cmb = generate_cmb_map(test_cls_tt, sigma_cmb_amp=fiducial_sigma_cmb, seed=42)

plt.figure(figsize=(5,5))
plt.imshow(test_cmb, cmap='coolwarm')
plt.title(f"Sample CMB Map (H0={fiducial_H0}, ombh2={fiducial_ombh2}, sigma={fiducial_sigma_cmb})")
plt.colorbar(label="$\mu K$")
# plt.savefig(os.path.join(OUTPUT_DIR,"sample_cmb_map.png"))
plt.show()
# print("CMB map generated and saved.")

# Test with varied parameters
varied_H0 = 80.0
varied_ombh2 = 0.03
varied_sigma_cmb = 0.5
test_cls_varied = get_camb_cls(H0=varied_H0, ombh2=varied_ombh2)
test_cmb_varied = generate_cmb_map(test_cls_varied, sigma_cmb_amp=varied_sigma_cmb, seed=43)

plt.figure(figsize=(5,5))
plt.imshow(test_cmb_varied, cmap='coolwarm')
plt.title(f"Sample CMB Map (H0={varied_H0}, ombh2={varied_ombh2}, sigma={varied_sigma_cmb})")
plt.colorbar(label="$\mu K$")
# plt.savefig(os.path.join(OUTPUT_DIR,"sample_cmb_map_varied.png"))
plt.show()
# print("Varied CMB map generated and saved.")

Data-Generation (Mixed)

In [ ]:
def generate_dataset_sample(index, seed_offset=0):
    """Generates one sample: dust_map, cmb_map, mixed_map, and cmb_params."""
    current_seed_dust = index + seed_offset 
    current_seed_cmb = index + seed_offset + NUM_SAMPLES_TO_GENERATE # ensure different seeds

    # 1. Get Dust map
    dust_map = load_or_generate_dust_map(index) # Use index for dust map selection if you have a list

    # 2. Sample CMB parameters from prior
    h0_sample = np.random.uniform(H0_PRIOR_MIN, H0_PRIOR_MAX)
    ombh2_sample = np.random.uniform(OMBH2_PRIOR_MIN, OMBH2_PRIOR_MAX)
    sigma_cmb_sample = np.random.uniform(SIGMA_CMB_PRIOR_MIN, SIGMA_CMB_PRIOR_MAX)
    
    cmb_params = {
        'H0': h0_sample,
        'ombh2': ombh2_sample,
        'sigma_cmb': sigma_cmb_sample
    }

    # 3. Generate CMB map
    cls_tt_sample = get_camb_cls(H0=h0_sample, ombh2=ombh2_sample)
    cmb_map = generate_cmb_map(cls_tt_sample, sigma_cmb_amp=sigma_cmb_sample, seed=current_seed_cmb)

    # 4. Create mixed map
    mixed_map = dust_map + cmb_map

    # 5. Save data
    enmap.write_map(os.path.join(OUTPUT_DIR, "cmb_maps", f"cmb_{index:04d}.fits"), cmb_map)
    enmap.write_map(os.path.join(OUTPUT_DIR, "mixed_maps", f"mixed_{index:04d}.fits"), mixed_map)
    np.save(os.path.join(OUTPUT_DIR, "params", f"params_{index:04d}.npy"), cmb_params)
    # Dust map is saved by load_or_generate_dust_map

    return dust_map, cmb_map, mixed_map, cmb_params

In [ ]:
print(f"\nGenerating {NUM_SAMPLES_TO_GENERATE} dataset samples...")
all_generated_data = []
for i in range(NUM_SAMPLES_TO_GENERATE):
    print(f"Generating sample {i+1}/{NUM_SAMPLES_TO_GENERATE}")
    d_map, c_map, m_map, params = generate_dataset_sample(i)
    all_generated_data.append({"dust": d_map, "cmb": c_map, "mixed": m_map, "params": params})
print("Dataset generation complete.")

# --- VI. Visualization of a Generated Sample ---
if NUM_SAMPLES_TO_GENERATE > 0:
    sample_idx_to_viz = 0
    data_viz = all_generated_data[sample_idx_to_viz]
    params_viz = data_viz["params"]

    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    im0 = axes[0].imshow(data_viz["dust"], cmap='magma')
    axes[0].set_title("Dust (Signal x)")
    fig.colorbar(im0, ax=axes[0], fraction=0.046, pad=0.04)

    im1 = axes[1].imshow(data_viz["cmb"], cmap='coolwarm')
    axes[1].set_title(f"CMB (Noise $\epsilon$)\nH0={params_viz['H0']:.1f}, ombh2={params_viz['ombh2']:.4f}, sigma={params_viz['sigma_cmb']:.2f}")
    fig.colorbar(im1, ax=axes[1], fraction=0.046, pad=0.04)

    im2 = axes[2].imshow(data_viz["mixed"], cmap='viridis')
    axes[2].set_title("Mixed (Observation y = x + $\epsilon$)")
    fig.colorbar(im2, ax=axes[2], fraction=0.046, pad=0.04)

    plt.tight_layout()
    # plt.savefig(os.path.join(OUTPUT_DIR, f"visualization_sample_{sample_idx_to_viz:04d}.png"))
    plt.show()
    # plt.close()

#### Training 

#### Denoising